In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

import argparse
import json
import gc
import datasets
import transformers
import torch
import evaluate
from tqdm import tqdm
import json
import numpy as np
from trl import PPOConfig, PPOTrainer, AutoModelForSeq2SeqLMWithValueHead
from peft import LoraConfig, TaskType, get_peft_model, PeftModelForSeq2SeqLM, PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

"""
Train fine tuned T5 model with Proximal Policy Optimization (PPO) algorithm.
"""
parser = argparse.ArgumentParser()
#parser.add_argument("--model_name", type=str, default="t5-small")
#parser.add_argument("--highlight", type=bool, default=True)
#args = parser.parse_args()


bertscore = evaluate.load("bertscore")
average_question_length = 10.0

HIGHLIGHT = True
TOKEN_QUESTION = "<question>"
TOKEN_END_QUESTION = "<question>"
TOKEN_CONTEXT = "<context>"
TOKEN_END_CONTEXT = "<context>"
TOKEN_ANSWER = "<answer>"
TOKEN_END_ANSWER = "<answer>"
HIGHLIGHT_ANSWER = "<hl>"
SPLIT_SEED = 42
NPROC = 32

model_name = "t5-small"
HIGHLIGHT = True
if HIGHLIGHT:
    model_name = f"{model_name}-hl"

/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-01 15:27:37,687] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
peft_config = LoraConfig(
    r=128,
    lora_alpha=64,
    lora_dropout=0.1,
    target_modules=["q", "v"],
)


#model = AutoModelForSeq2SeqLM.from_pretrained(f"./models/{model_name}/", device_map="auto")
#peft_model = PeftModelForSeq2SeqLM.from_pretrained(model, model_id=f"./models/{model_name}/", config=peft_config, device_map='auto', is_trainable=True)

ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(f"./models/{model_name}/", device_map="cuda:0")
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(f"./models/{model_name}/", device_map="cuda:0")
tokenizer = transformers.AutoTokenizer.from_pretrained(f"./models/{model_name}", model_max_length=512)
torch.cuda.empty_cache()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
tokenize_query = lambda e : tokenizer(e["query"], return_tensors='pt', padding=True, truncation=True).input_ids.squeeze().to('cuda:0')

def tokenize_query(e):
    return tokenizer(e["query"], return_tensors='pt', padding=True, truncation=True).input_ids.squeeze().to('cuda:0')

def get_inputs_target(e):
    answer_start = e["answers"]["answer_start"][0]
    # add highlight token to context
    ans_len = len(e["answers"]["text"][0])

    if HIGHLIGHT:
        e["context"] = (
            e["context"][:answer_start]
            + " "
            + HIGHLIGHT_ANSWER
            + " "
            + e["context"][answer_start : answer_start + ans_len]
            + " "
            + HIGHLIGHT_ANSWER
            + " "
            + e["context"][answer_start + ans_len :]
        )

    e['query'] = e.pop('context')

    return {
        # answer + context
        "query": f'generate question: {TOKEN_ANSWER} {e["answers"]["text"][0]} {TOKEN_END_ANSWER} {TOKEN_CONTEXT} {e["query"]} {TOKEN_END_CONTEXT}',
        # question
        "target": f'{TOKEN_QUESTION} {e["question"]} {TOKEN_END_QUESTION}',
        "answer": e["answers"]["text"][0],
    }


def preprocess_squad_dataset(dataset_name="squad", split="train"):
    dataset = datasets.load_dataset(dataset_name, split=split).shuffle(42).select(range(10000))  
    # Add question, answer and context tokens to dataset in a new column named text
    dataset = dataset.map(
        get_inputs_target,
        num_proc=16
    )
    return dataset

# Need to have training dataset aligned with PPO input format

train_dataset = preprocess_squad_dataset(dataset_name="squad", split="train") 

Map (num_proc=16): 100%|██████████| 10000/10000 [00:00<00:00, 12804.01 examples/s]


In [5]:
def preprocess_prediction(example):
    """
    this function will preprocess the prediction
    """
    return example.replace("<pad>", "").replace("<unk>", "").replace("</s>", "").replace("question>", "").replace("<question>", "").replace('<', "").strip()

def reward_model(example):
    """
    this function will return a reward function for PPO
    """
    try:
        context = example["query"]
        target = example["target"]
        answer = example["answer"]
        prediction = example["prediction"]
        prediction = [preprocess_prediction(pred) for pred in prediction] if isinstance(prediction, list) else preprocess_prediction(prediction)
        

        if isinstance(target, list):
            target = [preprocess_prediction(ans) for ans in answer]
        else:
            target = preprocess_prediction(target)

        
        

        reward = bertscore.compute(
            predictions=[prediction] if isinstance(prediction, str) else prediction,
            references=[target] if isinstance(target, str) else target,
            lang="en",
            model_type="bert-base-uncased",
        )["f1"][0]

        prediction = prediction[0] if isinstance(prediction, list) else prediction
        target = target[0] if isinstance(target, list) else target
        

        repetition_penalty = -5.0 if answer.lower() in prediction.lower() else 1.0
        question_word_penalty = -0.5 if target.split()[0].lower() != prediction.split()[0].lower() else 0.5
        question_length_penalty = -0.5 if len(target.split()) > 20 else 0.5

        reward = reward + (repetition_penalty + question_word_penalty  + question_length_penalty)
        # make it between 0 and 1
        reward = torch.nn.Sigmoid()(torch.tensor(reward))
        print(f"> Ans: {answer}, Q: {prediction}, P: {target}, R: {reward}")
        return reward

    except Exception as e:
        print("WARNING", e)
        return torch.tensor(0.0)

In [6]:
BATCH_SIZE = 16
config = PPOConfig(
    learning_rate=1e-5,
    log_with='tensorboard',
    project_kwargs={'logging_dir': f'./logs/{model_name}-ppo'},
    batch_size=BATCH_SIZE,
)

ppo_trainer = PPOTrainer(
    model=ppo_model,
    ref_model=ref_model,
    config=config,
    tokenizer=tokenizer,
)


batched_dataset = [[train_dataset[i] for i in range(j, min(j+BATCH_SIZE, len(train_dataset)))] for j in range(0, len(train_dataset), BATCH_SIZE)]
# remove last batch if it is not full
if len(batched_dataset[-1]) != BATCH_SIZE:
    batched_dataset.pop(-1)

for batch in tqdm(batched_dataset):
    query_tensors = [tokenizer(e["query"], return_tensors='pt', padding=True, truncation=True).input_ids.squeeze().to('cuda:0') for e in batch]

    response_tensors = ppo_trainer.generate(query_tensors, max_length=32, early_stopping=True)

    # put response_tensors into batch
    for i in range(len(batch)):
        batch[i]["prediction"] = tokenizer.decode(response_tensors[i], skip_special_tokens=True)

    pipe_outputs = [reward_model(e) for e in batch] 
    rewards = pipe_outputs

    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')

    print('-'.join('' for x in range(100)))

# save model
ppo_trainer.save_pretrained(f"./models/{model_name}-ppo/")

/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:254: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(
  0%|          | 0/625 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


> Ans: 84%, Q: How many Egyptians polled support the death penalty for those who leave Islam?, P: What percentage of Egyptians polled support death penalty for those leaving Islam?, R: 0.8697241544723511
> Ans: books, Q: What is the number of books sold per capita in Ann Arbor?, P: Ann Arbor ranks 1st among what goods sold?, R: 0.011874043382704258
> Ans: the executive, Q: Who controls the judiciary?, P: In developing countries, who makes most of the spending decisions?, R: 0.8336333632469177
> Ans: Anjiro, Q: What was the name of Francis's samurai?, P: Who impressed Xavier by taking notes in church?, R: 0.8101040720939636
> Ans: loops, Q: What are elements of the fundamental group represented by?, P: What represents elements of the fundamental group?, R: 0.9454767107963562
> Ans: 2.2 billion, Q: How many people are the population of the Commonwealth of Nations?, P: What is the population of the Commonwealth?, R: 0.8545026183128357
> Ans: Military Governor of the U.S. Occupation Zone, 

  0%|          | 1/625 [00:12<2:10:32, 12.55s/it]

objective/kl: 45.34385299682617
ppo/returns/mean: -2.016042470932007
ppo/policy/advantages_mean: -0.03571716696023941
---------------------------------------------------------------------------------------------------
> Ans: echiurans and sipunculan, Q: What are the polychaetes', P: What offshoots of polychaetes are unsegmented?, R: 0.9337579607963562
> Ans: Religion, Q: What is the important to the Tibetans?, P: What has a strong influence over all aspect of Tibetans lives?, R: 0.9346240758895874
> Ans: paralyzes muscles, Q: What does Clostridium tetani release a toxin?, P: What does the of toxin Clostridium tetani releases do?, R: 0.9425755143165588
> Ans: tituli, Q: What diocese of Rome is the term, P: The Church of England uses what term that is held by two senior members of the College of Minor Canons of St. Pauls Catherdral?, R: 0.6144264340400696
> Ans: CBS Television City, Q: Who is the title of the finals?, P: Where studio hosts the live final rounds on American Idol?, R: 0.82

  0%|          | 2/625 [00:22<1:52:49, 10.87s/it]

...

objective/kl: 46.11994934082031
ppo/returns/mean: -2.556302547454834
ppo/policy/advantages_mean: -0.026333559304475784
---------------------------------------------------------------------------------------------------
> Ans: Around the second century BC, Q: Around the second century BC, what were the first-known city-states in central Myanmar?, P: What period was the first known city states established in Myanmar?, R: 0.013850994408130646
> Ans: Louis Miriani, Q: What was the last mayor of Detroit?, P: Who was Detroit's last Republican mayor?, R: 0.8531922698020935
> Ans: 1948, Q: What did the National Security Council issue Directive 10/2 calling for covert action against USSR?, P: In what year was directive 10/2 issued?, R: 0.8359910845756531
> Ans: World Wealth Report, Q: What did the 2011, P: What organization ranked New Delhi 39th in economic activity in 2011?, R: 0.9202055931091309
> Ans: Latin Church, Q: What was the name of the earliest universities in Europe?, P: What church 

  5%|▍         | 30/625 [04:16<1:16:48,  7.74s/it]

objective/kl: 45.505855560302734
ppo/returns/mean: -2.624690532684326
ppo/policy/advantages_mean: -0.006543579511344433
---------------------------------------------------------------------------------------------------
> Ans: ASPEC, Q: What was the joint proposal of AT&T Bell Laboratories, Thomson Consumer Electronics, Fraunhofer Society and CNET, P: What was the name given to the proposal?, R: 0.9173706769943237
> Ans: five, Q: How many FA Cups did Arsenal win between 1988 and 2005?, P: Over a 17 year period which began in the late 80s, stretching five years into the new millenia, how many FA Cups did Arsenal win?, R: 0.6475770473480225
> Ans: City of Man, Q: What was the name of the city of God?, P: What does Civitas terrena mean?, R: 0.9235515594482422
> Ans: one of the world's poorest countries, Q: Liberia is what kind of country?, P: What is Liberia considered around the world economically?, R: 0.8291416764259338
> Ans: Shaye J. D. Cohen, Q: Who was the historian of mixed marriag

  5%|▍         | 31/625 [04:23<1:14:26,  7.52s/it]

objective/kl: 43.683555603027344
ppo/returns/mean: -2.618239402770996
ppo/policy/advantages_mean: -0.030731061473488808
---------------------------------------------------------------------------------------------------


In [ ]:
# save model 
